# Search Demo Corpus

We'll now continue the example with a recipe from the Vectara website where we search our employee handbook.

See here for the REST version of this: https://docs.vectara.com/docs/api-recipes

NB: try --no-cache-dir for pip install for new versions

In [1]:
%pip install -q vectara-skunk-client==0.2.5

Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging

logging.basicConfig(format='%(asctime)s %(name)-20s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')

In [3]:
from vectara.core import Factory

client = Factory().build()

12:20:07 +1100 Factory              INFO:initializing builder
12:20:07 +1100 Factory              INFO:Factory will load configuration from home directory
12:20:07 +1100 HomeConfigLoader     INFO:Loading configuration from users home directory [C:\Users\david]
12:20:07 +1100 HomeConfigLoader     INFO:Loading default configuration [default]
12:20:07 +1100 HomeConfigLoader     INFO:Parsing config
12:20:07 +1100 root                 INFO:We are processing authentication type [OAuth2]
12:20:07 +1100 OAuthUtil            INFO:Using provided OAuth2 URL [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
12:20:07 +1100 OAuthUtil            INFO:OAuth2 URL is [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
12:20:07 +1100 root                 INFO:initializing Client


In [4]:
admin_service = client.admin_service

corpora = admin_service.list_corpora("02-employee-handbook")

if len(corpora) >= 1:
    for corpus in corpora:
        admin_service.delete_corpus(corpus.id)
else:
    logging.info("No existing corpus with the name client-example")

12:20:07 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 12:20:07.059216
12:20:07 +1100 OAuthUtil            INFO:First time requesting token, authenticating
12:20:08 +1100 OAuthUtil            INFO:Received OAuth token, will expire [12/29/2023, 13:20:08]
12:20:08 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703816408]
12:20:08 +1100 RequestUtil          INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
12:20:09 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 12:20:09.125745
12:20:09 +1100 OAuthUtil            INFO:Expiry            2023-12-29 13:20:08
12:20:09 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703816408]
12:20:09 +1100 RequestUtil          INFO:URL for operation delete-corpus is: https://api.vectara.io/v1/delete-corpus


## Create Our Demo Corpus
We now create our new corpus called, "02-employee-handbook" where we'll upload our documents for this test.

In [5]:
create_corpus_result = admin_service.create_corpus("02-employee-handbook", description="Example Corpus for use from Jupyter")
logging.info(f"New corpus created {create_corpus_result}")
corpus_id = create_corpus_result.corpusId

12:20:11 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 12:20:11.106810
12:20:11 +1100 OAuthUtil            INFO:Expiry            2023-12-29 13:20:08
12:20:11 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703816408]
12:20:11 +1100 RequestUtil          INFO:URL for operation create-corpus is: https://api.vectara.io/v1/create-corpus
12:20:13 +1100 AdminService         INFO:Created new corpus with 122
12:20:13 +1100 root                 INFO:New corpus created CreateCorpusResponse(corpusId=122, status=Status(code=<StatusCode.OK: 0>, statusDetail='Corpus Created'))


## Upload Handbook to Corpus
We now upload our PDF to the corpus which will parse and then encode the document, storing the embeddings in our vector database.

In [6]:
from pathlib import Path

handbook_path = Path("resources/vectara_employee_handbook.pdf")
indexer_service = client.indexer_service
indexer_service.upload(corpus_id, handbook_path)

12:20:13 +1100 IndexerService       INFO:Headers: {"c": "1623270172", "o": "122"}
12:20:13 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 12:20:13.091609
12:20:13 +1100 OAuthUtil            INFO:Expiry            2023-12-29 13:20:08
12:20:13 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703816408]
vectara_employee_handbook.pdf: 52.7kB [00:03, 17.1kB/s]                                                                


UploadDocumentResponse(response=UploadDocumentResponseInner(status=None, quotaConsumed=StorageQuota(numChars='9215', numMetadataChars='4903')), document=None)

In [7]:
query_service = client.query_service
query = "At Vectara, Can I bring any birds to the Vectara Office?"
response = query_service.query(query, corpus_id)

12:20:16 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 12:20:16.273401
12:20:16 +1100 OAuthUtil            INFO:Expiry            2023-12-29 13:20:08
12:20:16 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703816408]
12:20:16 +1100 RequestUtil          INFO:URL for operation query is: https://api.vectara.io/v1/query


## Show Result in Markdown
We'll now use the utility method "renderMarkdown" to render the Python dict

In [8]:
from vectara.util import render_markdown
from IPython.display import display, Markdown
rendered = render_markdown(query, response)
display(Markdown(rendered))


# Query: At Vectara, Can I bring any birds to the Vectara Office?

At Vectara, you are allowed to bring birds to the office [7.2.3]. While common household pets like cats and dogs are not permitted [7.2.1], Vectara embraces the unusual and exotic [7.2.2]. However, it is important to note that the employee handbook contains a pet policy [7.2]. Unfortunately, the specific details regarding bird types or any restrictions are not mentioned in the available search results.

 1. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): 7.2.3: The Avian Exception - Birds of a Feather <b>At Vectara, we believe in soaring to new heights, and what better way to embody that spirit than
by allowing birds into our workspace?</b> Yes, you read that right! *score: 0.89985037*
 2. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): 7.2.1: The Usual Suspects - Cats & Dogs <b>We regret to inform you that common household pets such as cats and dogs are not allowed on
the Vectara campuses.</b> Why, you ask? Because at Vectara, we're all about breaking the mold and
challenging the status quo. *score: 0.8140942*
 3. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): Vectara Employee Handbook <b>This employee handbook covers all the essential information to make you successful at
Vectara.</b> 7.2 - Company Pet Policy *score: 0.8071276*
 4. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): While we adore cats and dogs, we must respectfully ask that they remain at home, where they
can continue their secret missions, philosophical ponderings, and adorable antics. <b>At Vectara,
we're forging a new path, one that requires us to leave behind the familiar and embrace the wild
unknown.</b> Rest assured, your feline and canine companions will be cheering you on from the
comfort of their favorite sunlit windows and cozy dog beds. *score: 0.8032716*
 5. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): 7.2.2: The Exotic Exceptions - Unleashing the Unusual <b>At Vectara, we pride ourselves on thinking outside the box, and our pet policy is no exception.</b> While common household pets like cats and dogs are not permitted, we welcome a select group
of exotic creatures that reﬂect our daring and innovative spirit. *score: 0.796098*
 6. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): We regret to inform you that common household pets such as cats and dogs are not allowed on
the Vectara campuses. <b>Why, you ask? Because at Vectara, we're all about breaking the mold and
challenging the status quo.</b> Cats and dogs are wonderful, but we believe in taking a walk on the
wild side. *score: 0.7942082*
 7. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): Adoption Programs <b>The Conservation Connection: Vectara is committed to wildlife conservation.</b> Our exotic pet
policy is aligned with global conservation efforts, and we actively support initiatives to protect
endangered species. *score: 0.79056066*
 8. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): Employee Handbook - Company Pet Policy <b>Vectara Employee Handbook</b> This employee handbook covers all the essential information to make you successful at
Vectara. *score: 0.7843369*
 9. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): On-site Mini Zoo <b>●   The Vectara Zoo
               ○   Our on-site mini-zoo is home to these extraordinary animals.</b> It's a place of
                       inspiration, relaxation, and education. *score: 0.76677066*
 10. **Employee Handbook - Company Pet Policy** (vectara_employee_handbook.pdf): Furthermore, we've consulted with leading pet psychologists, animal behaviorists, and even a
few wizards to understand the complex dynamics between humans and their four-legged
companions. <b>The conclusion? Cats and dogs are simply too mainstream for Vectara's
avant-garde environment.</b> They're playing chess while we're orchestrating interdimensional
quantum maneuvers. *score: 0.7343332*


In [9]:
print(corpus_id)

122
